# Update Summary Report HTML version

## Prerequsite 

In [ ]:

# SET log level
# logging
import logging
logger = logging.getLogger()
logger.setLevel(logging.WARN)
logging.info("Prerequisite")

import ipywidgets as widgets
from IPython.display import display, HTML
import os
import configparser
config = configparser.ConfigParser()
config.read('data.conf')
es_host = config.get("data", 'es_host') 
es_port = config.get("data", 'es_port')
es_user = config.get("data", 'es_user') 
es_password = config.get("data", 'es_password')
grafana_url = config.get("data", 'grafana_url')
os.environ['network_speed'] = config.get("data", 'network_speed')
os.environ['fetch_ocp_versions_days'] = config.get("data", 'fetch_ocp_versions_days')
os.environ['filter_kind'] = config.get("data", 'filter_kind')

from benchmark_runner.common.elasticsearch.elasticsearch_operations import ElasticSearchOperations
from benchmark_runner.jupyterlab.templates.summary_report.summary_report_widgets import SummaryReportWidgets

elasticsearch = ElasticSearchOperations(es_host=es_host, es_port=es_port, es_user=es_user, es_password=es_password)
summary_report_widget = SummaryReportWidgets(elasticsearch=elasticsearch)

# Default 365 days for ocp version fetch days 
summary_report_widget.set_fetch_ocp_version_days(num_fetch_days=int(os.environ['fetch_ocp_versions_days']))


## Update file name with 2 last versions comparison

In [ ]:
import os
versions = summary_report_widget.get_two_last_major_versions()
os.rename('summary_report.html', f'summary_report_{versions[0]}_{versions[1]}.html')